In [514]:
import numpy as np

In [515]:
class Entity:
    def __init__(self, type, Node1, Node2, physicalValue, initialValue):
        self.type = type
        self.Node1 = Node1
        self.Node2 = Node2
        self.physicalValue = physicalValue
        self.initialValue = initialValue
    def __str__(self):
        return f"Type: {self.type},\nNode1: {self.Node1},\nNode2: {self.Node2},\nPhysical Value: {self.physicalValue},\nInitial Value: {self.initialValue}\n"

In [516]:
class Simulation:
    def __init__(self, fileNameNum = ''):
        self.simTime = 0
        self.iterations = 0
        
        self.entities = []
        
        self.numberOfVoltageSources = 0
        self.numberOfInuductors = 0
        
        self.nodes = []
        self.numberOfNodes = 0
        
        self.A = 0
        self.Z = 0
        self.currentZ = 0
        self.X = 0
        self.XLabels = 0
        
        self.simValues = []
        
        self.fileNameNum = fileNameNum
        self.readFile('./testcases/'+fileNameNum+'.txt')
        
    def readFile(self, fileName):
        #Opening the file
        with open(fileName, 'r') as file:
            lines = file.readlines()
            #Getting the first two lines
            self.simTime = float(lines[0])
            self.iterations = int(lines[1])
            
        for line in lines[2:-1]:
            #The rest are inputs so we split them in order to get each entites information
            data = line.split()
            componentType = data[0]
            node1 = int(data[1][1:])
            node2 = int(data[2][1:])
            physicalValue = float(data[3])
            initialValue = float(data[4])
            #We add it into our entities list
            newEntity = Entity(componentType,node1,node2,physicalValue,initialValue)
            self.entities.append(newEntity)
            
            #We count the number of Voltage sources and inductors to be used later
            if componentType == 'Vsrc':
                self.numberOfVoltageSources += 1
            if componentType == 'I':
                self.numberOfInuductors += 1
            #We make a list of nodes and get their count
            if (node1 not in self.nodes):
                self.nodes.append(node1)
            if (node2 not in self.nodes):
                self.nodes.append(node2)
        #We do not take in consideration the ground node
        self.nodes.remove(0)
        self.numberOfNodes = len(self.nodes)
            
    def calculateAMatrix(self):
        #Initialize all the matrices according to the new acquired sizes
        G = np.zeros((self.numberOfNodes, self.numberOfNodes))
        B = np.zeros((self.numberOfNodes,self.numberOfVoltageSources + self.numberOfInuductors))
        C = np.zeros((self.numberOfVoltageSources + self.numberOfInuductors,self.numberOfNodes))
        D = np.zeros((self.numberOfVoltageSources + self.numberOfInuductors,self.numberOfVoltageSources + self.numberOfInuductors))
        #Filling the A matrix according the slides
        for node in self.nodes:
            for entity in self.entities:
                #Case of Resistance
                if entity.type == 'R' and entity.Node1 == node:
                    G[node-1,node-1] += 1/entity.physicalValue
                    if entity.Node2 != 0:
                        G[node-1,entity.Node2-1] += -1/entity.physicalValue
                if entity.type == 'R' and entity.Node2 == node:
                    G[node-1,node-1] += 1/entity.physicalValue
                    if entity.Node1 != 0:
                        G[node-1,entity.Node1-1] += -1/entity.physicalValue
                #Case of Capacitor
                if entity.type == 'C' and entity.Node1 == node:
                    G[node-1,node-1] += entity.physicalValue/self.simTime
                    if entity.Node2 != 0:
                        G[node-1,entity.Node2-1] += -entity.physicalValue/self.simTime
                if entity.type == 'C' and entity.Node2 == node:
                    G[node-1,node-1] += entity.physicalValue/self.simTime
                    if entity.Node1 != 0:
                        G[node-1,entity.Node1-1] += -entity.physicalValue/self.simTime
        #Filling the B matrix according to the slides starting with voltage sources then with the inductors
        currentVS = 0
        for entity in self.entities:
            if (entity.type == 'Vsrc'):
                if entity.Node1 != 0:
                    B[entity.Node1-1,currentVS] = 1
                if entity.Node2 != 0:
                    B[entity.Node2-1,currentVS] = -1
                currentVS += 1
        #After we finish the voltage sources we fill the data of the inducotrs
        for entity in self.entities:
            if entity.type == 'I':
                if entity.Node1 != 0:
                    B[entity.Node1-1,currentVS] = 1
                if entity.Node2 != 0:
                    B[entity.Node2-1,currentVS] = -1
                #Filling the D matrix also
                D[currentVS,currentVS] = -entity.physicalValue/self.simTime
                currentVS += 1
        #The C matrix is the transpose of B matrix
        C = np.transpose(B)
        #We gather all of the matrices to get the A matrix
        topHalf = np.hstack((G,B))
        bottomHalf = np.hstack((C,D))
        self.A = np.vstack((topHalf,bottomHalf))
        print("A matrix: ")
        print(self.A)
        
    def identifyXMatrix(self):
        #This function on gives us the labels of each entity that we are going to display in the output file
        v=[]
        i=[]
        #Adding the voltages at each node
        for node in self.nodes:
            v.append("V"+str(node))
        currentVS = 0
        currentI = 0
        #Voltage sources first
        for entity in self.entities:
            if entity.type == 'Vsrc':
                i.append("I_Vsrc" + str(currentVS))
                currentVS += 1
        #Then inductors
        for entity in self.entities:
            if entity.type == "I":
                i.append("I_L" + str(currentI))
                currentI += 1
        #Grouping them together
        self.XLabels = v + i
        
    def calculateZMatrix(self):
        #Initialzing the I and J matrices according to the new sizes and filling them according to slides
        i = np.zeros((self.numberOfNodes,1))
        j = np.zeros((self.numberOfVoltageSources + self.numberOfInuductors,1))
        currentVS = 0
        #We add the current sources in the i matrix and we add the voltage sources in the j matrix and then we add them together
        for entity in self.entities:
            if entity.type == 'Isrc':
                if entity.Node1 != 0:
                    i[entity.Node1-1,0] += entity.physicalValue
                if entity.Node2 != 0:
                    i[entity.Node2-1,0] -= entity.physicalValue
            elif entity.type == 'Vsrc':
                j[currentVS,0] = entity.physicalValue
                currentVS += 1
        self.Z = np.vstack((i,j))
    
    def updateInitialValues(self):
        #This function is called in every interation to update the inital values of capacitors and inductors if exist
        currentI = 0
        for entity in self.entities:
            newValue = 0
            #Capacitor
            if entity.type == "C":
                if entity.Node1 != 0:
                    newValue += self.X[entity.Node1-1,0]
                if entity.Node2 !=0 :
                    newValue -= self.X[entity.Node2-1,0]
                entity.initialValue = newValue
            #Inductor
            if entity.type == "I":
                entity.initialValue = self.X[self.numberOfNodes + self.numberOfVoltageSources + currentI]
                currentI += 1
        
    def updateZMatrix(self):
        #This function updates the Z matrix after calculating the new inital values
        #It is important to note that in every interation we update the original Z matrix and NOT the previous one
        currentI = 0
        #Resetting the Z matrix
        self.currentZ = self.Z.copy()
        #Calculating the new initial values of capacitors and inductors
        for entity in self.entities:
            if entity.type == 'C':
                if entity.Node1 != 0:
                    self.currentZ[entity.Node1-1,0] += (entity.physicalValue/self.simTime) * entity.initialValue
                if entity.Node2 != 0:
                    self.currentZ[entity.Node2-1,0] -= (entity.physicalValue/self.simTime) * entity.initialValue
            if entity.type == 'I':
                self.currentZ[self.numberOfNodes + self.numberOfVoltageSources + currentI,0] += - (entity.physicalValue / self.simTime) * entity.initialValue
                currentI += 1
        #print("Z",self.currentZ)
        
    def calculateX(self):
        #This function is used to calculate the X matrix using the rule (A^-1)z
        self.X = np.dot(np.linalg.inv(self.A),self.currentZ)
        self.simValues.append(self.X)
        #print("X",self.X)
        
    def simulate(self):
        #This function groups all the previous functions and uses them in order
        self.calculateAMatrix()
        self.calculateZMatrix()
        self.identifyXMatrix()
        for i in range(self.iterations):
            self.updateZMatrix()
            self.calculateX()
            self.updateInitialValues()
        print("Final X: ")
        print(self.X)
            
    def createOutputFile(self):
        #Start initially with the sim time
        currentSimTime = self.simTime
        #Creating output file
        with open('output'+self.fileNameNum+'.txt', 'w') as file:
            #Seperating each component
            for i in range(len(self.XLabels)):
                file.write(self.XLabels[i] + '\n')
                #Writing its value in each of the operations
                for arr in self.simValues:
                    file.write(str(round(currentSimTime,3)) + " ")
                    currentSimTime += self.simTime
                    file.write(str(round(arr[i][0],12)) + '\n')
                file.write('\n')
                #Returning to original for the next component
                currentSimTime = self.simTime

In [517]:
fileName = input("Enter file name: ")
sim = Simulation(fileName)
sim.simulate()
sim.createOutputFile()

A matrix: 
[[ 12.  -1.   0.]
 [ -1.   1.   1.]
 [  0.   1. -10.]]
Final X: 
[[0.1328971 ]
 [0.10003651]
 [1.03286059]]


C:\Users\danie_dnkrtc3\AppData\Local\Temp\ipykernel_22812\446344701.py:184: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  self.currentZ[self.numberOfNodes + self.numberOfVoltageSources + currentI,0] += - (entity.physicalValue / self.simTime) * entity.initialValue
